In [39]:
import json

# Load data from JSON
with open('apiResponse/all_responses_200_sample.json', 'r', encoding='utf-8') as file:
    api_data = json.load(file)

with open('Samples/sample_siirtokarjalaiset_annotated.json', 'r', encoding='utf-8') as file:
    hand_data = json.load(file)



In [40]:

from fuzzywuzzy import fuzz

def parse_response(response_str):
    lines = response_str.split('\n')
    parsed_response = {}
    for line in lines:
        key, _, value = line.partition(': ')
        parsed_response[key] = value.strip() if value.strip() else None
    return parsed_response


# Checking how similar the words are
def are_similar(str1, str2, threshold=60, context=None):
    similarity = fuzz.token_set_ratio(str1, str2)
    is_similar = similarity > threshold
    
    # Check if similarity is below 100% and store it if so
    if is_similar and similarity < 100:
        store_not_exact_matches(str1, str2, similarity, context)
        
    return is_similar


def store_not_exact_matches(str1, str2, similarity, context):
    data = {
        "string_1": str1,
        "string_2": str2,
        "similarity": similarity,
        "context": context  # This will provide additional information
    }
    with open("non_exact_matches.json", "a", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)
        file.write(",\n")






In [41]:
def compare_values(api_values, annotated_values, context):
    api_list = api_values.lower().split(', ') if api_values else []
    annotated_list = annotated_values.lower().split(', ') if annotated_values else []
    
    matches = set()
    mismatches = set(api_list).union(set(annotated_list))  
    
    for api_val in api_list:
        for ann_val in annotated_list:
            if are_similar(api_val, ann_val, context=context):
                matches.add(api_val)
                mismatches.discard(api_val)
                mismatches.discard(ann_val)
    
    return matches, mismatches


# Parse JSON strings
api_responses = api_data
hand_annotated = hand_data
# Loop over all elements in api_responses and hand_annotated to compare them
results = []
total_matches = 0
total_mismatches = 0

for api_resp, hand_ann in zip(api_responses, hand_annotated):
    parsed_api_response = parse_response(api_resp['api_response'])
    
    comparison_results = {
        "index": hand_ann['index'],
        "person_name": hand_ann['primary_person_name'],
        "spouse_name": hand_ann['spouse_name'],
        "detail": []
    }
    
    for key in ["person_hobbies", "person_social_orgs", "spouse_hobbies", "spouse_social_orgs"]:
        split_keys = key.split("_")
        api_key = split_keys[0].capitalize() + "".join(word.capitalize() for word in split_keys[1:])
        
        # Safely get the index values for context
        api_person_index = api_resp.get('person_index', None)
        annotated_person_index = hand_ann.get('index', None)

        # Build the context
        context = {
            "api_person_index": api_person_index +1,
            "annotated_person_index": annotated_person_index
        }
        
        matches, mismatches = compare_values(parsed_api_response.get(api_key, ""), hand_ann[key], context)
        
        detail = {
            "type": key,
            "matches": list(matches),
            "mismatches": list(mismatches)
        }
        comparison_results["detail"].append(detail)
        
        total_matches += len(matches)
        total_mismatches += len(mismatches)
    
    results.append(comparison_results)

output_json = json.dumps(results, indent=4, ensure_ascii=False)

# To store the results in a file:
with open("matches_results.json", "w") as file:
    file.write(output_json)

# Printing total matches and mismatches
print(f"Total Matches: {total_matches}")
print(f"Total Mismatches: {total_mismatches}")

# Calculating and printing the match percentage
total_comparisons = total_matches + total_mismatches
if total_comparisons > 0: 
    match_percentage = (total_matches / total_comparisons) * 100
    print(f"Match Percentage: {match_percentage:.2f}%")
else:
    print("No comparisons were made (Total Comparisons: 0).")

Total Matches: 361
Total Mismatches: 463
Match Percentage: 43.81%


In [42]:
def compare_values(api_dict, hand_ann_dict):
    api_name = api_dict["api_response"].split("PersonName:")[1].split("\n")[0].strip()
    api_hobbies = api_dict["api_response"].split("PersonHobbies:")[1].split("\n")[0].strip().split(', ')
    api_social_orgs = api_dict["api_response"].split("PersonSocialOrgs:")[1].split("\n")[0].strip().split(', ')
    api_spouse_hobbies = api_dict["api_response"].split("SpouseHobbies:")[1].split("\n")[0].strip().split(', ')
    api_spouse_social_orgs = api_dict["api_response"].split("SpouseSocialOrgs:")[1].split("\n")[0].strip().split(', ')

    hand_name = hand_ann_dict["primary_person_name"]
    hand_hobbies = hand_ann_dict["person_hobbies"].split(', ')
    hand_social_orgs = hand_ann_dict["person_social_orgs"].split(', ')
    hand_spouse_hobbies = hand_ann_dict["spouse_hobbies"].split(', ')
    hand_spouse_social_orgs = hand_ann_dict["spouse_social_orgs"].split(', ')

    # Assuming are_similar function is predefined

    def calculate_metrics(api_values, hand_values):
        TP = set()
        FP = set(api_values)  # Temporarily assume all api_values are FP
        FN = set(hand_values)  # Temporarily assume all annotated_values are FN
        
        for api_val in api_values:
            for ann_val in hand_values:
                if are_similar(api_val, ann_val):
                    TP.add(api_val)  # Add to True Positives
                    FP.discard(api_val)  # Remove from False Positives
                    FN.discard(ann_val)  # Remove from False Negatives
                    
        precision = len(TP) / (len(TP) + len(FP)) if TP or FP else 0
        recall = len(TP) / (len(TP) + len(FN)) if TP or FN else 0
        f_score = (2 * precision * recall) / (precision + recall) if precision + recall > 0 else 0
    
        return TP, FP, FN, precision, recall, f_score

    # Compute metrics for person and spouse separately
    person_metrics = calculate_metrics(api_social_orgs, hand_social_orgs)
    person_hobbies = calculate_metrics(api_hobbies, hand_hobbies)
    spouse_metrics = calculate_metrics(api_spouse_social_orgs, hand_spouse_social_orgs)
    spouse_hobbies = calculate_metrics(api_spouse_hobbies, hand_spouse_hobbies)

    

    return person_metrics, spouse_metrics, person_hobbies, spouse_hobbies

    



api_responses = api_data  # This should be your API data
hand_annotated = hand_data  # This should be your hand annotated data

results = []


# Assume api_responses and hand_annotated are lists of strings for this example. 
# Modify as per your actual data structure.
total_person_TP = total_person_FP = total_person_FN = 0
total_spouse_TP = total_spouse_FP = total_spouse_FN = 0
total_person_hobbies_TP = total_person_hobbies_FP = total_person_hobbies_FN = 0
total_spouse_hobbies_TP = total_spouse_hobbies_FP = total_spouse_hobbies_FN = 0


for index, (api_resp, hand_ann) in enumerate(zip(api_responses, hand_annotated)):
    # Call compare_values and unpack all four return values here:
    person_metrics, spouse_metrics, person_hobbies_metrics, spouse_hobbies_metrics = compare_values(api_resp, hand_ann)

    total_person_TP += len(person_metrics[0])
    total_person_FP += len(person_metrics[1])
    total_person_FN += len(person_metrics[2])

    total_spouse_TP += len(spouse_metrics[0])
    total_spouse_FP += len(spouse_metrics[1])
    total_spouse_FN += len(spouse_metrics[2])
    
    # Add handling for person_hobbies_metrics and spouse_hobbies_metrics
    # For example:
    total_person_hobbies_TP += len(person_hobbies_metrics[0])
    total_person_hobbies_FP += len(person_hobbies_metrics[1])
    total_person_hobbies_FN += len(person_hobbies_metrics[2])

    total_spouse_hobbies_TP += len(spouse_hobbies_metrics[0])
    total_spouse_hobbies_FP += len(spouse_hobbies_metrics[1])
    total_spouse_hobbies_FN += len(spouse_hobbies_metrics[2])

    results.append({
    "person_metrics": {
        "index": hand_ann['index'],
        "person_name": hand_ann['primary_person_name'],
        "true_positives": list(person_metrics[0]),
        "false_positives": list(person_metrics[1]),
        "false_negatives": list(person_metrics[2]),
        "precision": person_metrics[3],
        "recall": person_metrics[4],
        "f_score": person_metrics[5]
    },
    "person_hobbies_metrics": {
        "true_positives": list(person_hobbies_metrics[0]),
        "false_positives": list(person_hobbies_metrics[1]),
        "false_negatives": list(person_hobbies_metrics[2]),
        "precision": person_hobbies_metrics[3],
        "recall": person_hobbies_metrics[4],
        "f_score": person_hobbies_metrics[5]
    },
    "spouse_metrics": {
        "true_positives": list(spouse_metrics[0]),
        "false_positives": list(spouse_metrics[1]),
        "false_negatives": list(spouse_metrics[2]),
        "precision": spouse_metrics[3],
        "recall": spouse_metrics[4],
        "f_score": spouse_metrics[5]
    },
    "spouse_hobbies_metrics": {
        "true_positives": list(spouse_hobbies_metrics[0]),
        "false_positives": list(spouse_hobbies_metrics[1]),
        "false_negatives": list(spouse_hobbies_metrics[2]),
        "precision": spouse_hobbies_metrics[3],
        "recall": spouse_hobbies_metrics[4],
        "f_score": spouse_hobbies_metrics[5]
    }
})


# Constructing a summary of results for final output
# Constructing a summary of results for final output
person_precision = total_person_TP / (total_person_TP + total_person_FP) if total_person_TP + total_person_FP > 0 else 0
person_recall = total_person_TP / (total_person_TP + total_person_FN) if total_person_TP + total_person_FN > 0 else 0
spouse_precision = total_spouse_TP / (total_spouse_TP + total_spouse_FP) if total_spouse_TP + total_spouse_FP > 0 else 0
spouse_recall = total_spouse_TP / (total_spouse_TP + total_spouse_FN) if total_spouse_TP + total_spouse_FN > 0 else 0

# Calculate overall metrics
overall_TP = total_person_TP + total_spouse_TP
overall_FP = total_person_FP + total_spouse_FP
overall_FN = total_person_FN + total_spouse_FN

overall_precision = overall_TP / (overall_TP + overall_FP) if (overall_TP + overall_FP) > 0 else 0
overall_recall = overall_TP / (overall_TP + overall_FN) if (overall_TP + overall_FN) > 0 else 0
overall_f_score = (2 * overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0

# Round to 3 decimal places
overall_precision = round(overall_precision, 3)
overall_recall = round(overall_recall, 3)
overall_f_score = round(overall_f_score, 3)

summary = {
    "person": {
        "total_true_positives": total_person_TP,
        "total_false_positives": total_person_FP,
        "total_false_negatives": total_person_FN,
        "precision": person_precision,
        "recall": person_recall,
        "f_score": (2 * person_precision * person_recall) / (person_precision + person_recall) if person_precision + person_recall > 0 else 0
    },
    "spouse": {
        "total_true_positives": total_spouse_TP,
        "total_false_positives": total_spouse_FP,
        "total_false_negatives": total_spouse_FN,
        "precision": spouse_precision,
        "recall": spouse_recall,
        "f_score": (2 * spouse_precision * spouse_recall) / (spouse_precision + spouse_recall) if spouse_precision + spouse_recall > 0 else 0
    }, 
    "overall": {
        "total_true_positives": overall_TP,
        "total_false_positives": overall_FP,
        "total_false_negatives": overall_FN,
        "precision": overall_precision,
        "recall": overall_recall,
        "f_score": overall_f_score
    }
}
for key in summary:
    summary[key]['precision'] = round(summary[key]['precision'], 3)
    summary[key]['recall'] = round(summary[key]['recall'], 3)
    summary[key]['f_score'] = round(summary[key]['f_score'], 3)


# Final JSON output to include both the detailed results and the summary
output_data = {
    "summary": summary,
    "results": results
}

# Storing in JSON format
output_json = json.dumps(output_data, indent=4, ensure_ascii=False)

# To store the results in a file:
with open("precision_recall_results.json", "w", encoding="utf-8") as file:
    file.write(output_json)

# Printing total matches and mismatches
print(f"--- Person ---")
print(f"Total True Positives: {total_person_TP}")
print(f"Total False Positives: {total_person_FP}")
print(f"Total False Negatives: {total_person_FN}")
print(f"Precision: {summary['person']['precision']:.2f}")
print(f"Recall: {summary['person']['recall']:.2f}")
print(f"\n--- Spouse ---")
print(f"Total True Positives: {total_spouse_TP}")
print(f"Total False Positives: {total_spouse_FP}")
print(f"Total False Negatives: {total_spouse_FN}")
print(f"Precision: {summary['spouse']['precision']:.2f}")
print(f"Recall: {summary['spouse']['recall']:.2f}")
print(f"\n--- Overall ---")
print(f"Total True Positives: {overall_TP}")
print(f"Total False Positives: {overall_FP}")
print(f"Total False Negatives: {overall_FN}")
print(f"Precision: {overall_precision:.3f}")
print(f"Recall: {overall_recall:.3f}")
print(f"F-score: {overall_f_score:.3f}")

--- Person ---
Total True Positives: 132
Total False Positives: 157
Total False Negatives: 191
Precision: 0.46
Recall: 0.41

--- Spouse ---
Total True Positives: 65
Total False Positives: 187
Total False Negatives: 205
Precision: 0.26
Recall: 0.24

--- Overall ---
Total True Positives: 197
Total False Positives: 344
Total False Negatives: 396
Precision: 0.364
Recall: 0.332
F-score: 0.347
